<a href="https://colab.research.google.com/github/yancheng-du/lyrics_generator/blob/main/Copy_of_TaylorSwift_lyrics_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Taylor Swift Lyrics Generator**
<!-- ![](https://cdn-images-1.medium.com/max/1800/1*VSXmKKJJZFlUVVZWBwbXVg.jpeg) -->

A few days ago, I started to learn LSTM RNN (Long Short Term Memory Recurrent Neural Networks), and I thought that it would be a good idea if I make a project using it.

There is a multitude of applications of LSTM RNN, I decided to go with natural language generation because it will be a good opportunity to learn how to process text data, and it will be entertaining to see texts generated by neural networks, so I got this idea about generating Taylor Swift lyrics.


## **What is LSTM Recurrent Neural Networks ?**
If you don't know, LSTM recurrent neural networks are networks with loops in them, allowing information to persist, and they have a special type of nodes called LSTM(Long Short Term Memory).

LSTM unit is composed of a cell, an input gate, an output gate and a forget gate. The cell remembers values over arbitrary time intervals and the three gates regulate the flow of information into and out of the cell.
If you want to know more about  LSTM Recurrent Neural Networks visit :
[Understanding LSTM Networks](https://goo.gl/dgmxQm) or [Long short-term memoryt](https://goo.gl/Dc7kHF)


## LSTM Recurrent Neural Networks Applications 
LSTM Recurrent Neural Networks are used in many applicattions , the following are the most popular ones :



*   Language modeling
*   Text classification
*   Dialog systems
*   **Natural language generation**

[More applications](https://goo.gl/eT3bMm)


Now, after we learned some essential information about LSTM and RNN , we will start implementing the idea (Taylor Swift Lyrics Generator) 

I will use two ways to build the model :
* From scratch
* Using a Python module called [textgenrnn](https://goo.gl/E7szXj)

## Process The Dataset 
To train the LSTM model we need a dataset of Taylor songs' lyrics.
After searching for it, I found [this great dataset](https://goo.gl/3oUpMG) in Kaggle .

**Let's take a look at it :**


first, import all the needed libraries for our project:

In [ ]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.4.3)


In [ ]:
from keras.layers import LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
# Import the dependencies
import numpy as np
import pandas as pd
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


**Load the dataset :**

In [ ]:
#Load the dataset
url = 'https://raw.githubusercontent.com/yancheng-du/lyrics_generator/main/taylor_swift_lyrics.csv'
dataset = pd.read_csv(url, encoding = "latin1")


In [ ]:
dataset.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [ ]:
dataset.describe()

,track_n,line,year
count,4862.000000,4862.000000,4862.000000
mean,8.216989,28.426573,2011.882764
std,4.696379,18.343649,3.571447
min,1.000000,1.000000,2006.000000
25%,4.000000,13.000000,2010.000000
50%,8.000000,26.000000,2012.000000
75%,12.000000,41.000000,2014.000000
max,19.000000,101.000000,2017.000000


From what we see , we need to concatenate the lines of each song to get each song by its own in one string.

In [14]:
def processFirstLine(lyrics, songID, songName, row):
    lyrics.append(row['lyric'] + '\n')
    songID.append( row['year']*100+ row['track_n'])
    songName.append(row['track_title'])
    return lyrics,songID,songName

In [15]:
# define empty lists for the lyrics , songID , songName 
lyrics = []
songID = []
songName = []

# songNumber indicates the song number in the dataset
songNumber = 1

# i indicates the song number
i = 0
isFirstLine = True

# Iterate through every lyrics line and join them together for each song independently 
for index,row in dataset.iterrows():
    if(songNumber == row['track_n']):
        if (isFirstLine):
            lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
            isFirstLine = False
        else :
            #if we still in the same song , keep joining the lyrics lines    
            lyrics[i] +=  row['lyric'] + '\n'
    #When it's done joining a song's lyrics lines , go to the next song :    
    else :
        lyrics,songID,songName = processFirstLine(lyrics,songID,songName,row)
        songNumber = row['track_n']
        i+=1


In [16]:

# Define a new pandas DataFrame to save songID , songName , Lyrics in it to use them later
lyrics_data = pd.DataFrame({'songID':songID, 'songName':songName, 'lyrics':lyrics })

lyrics_data[:5]

,songID,songName,lyrics
0,200601,Tim McGraw,He said the way my blue eyes shined\nPut those...
1,200602,Picture To Burn,"State the obvious, I didn't get my perfect fan..."
2,200603,Teardrops On My Guitar,Drew looks at me\nI fake a smile so he won't s...
3,200604,A Place In This World,"I don't know what I want, so don't ask me\nCau..."
4,200605,Cold as You,You have a way of coming easily to me\nAnd whe...


Now save the lyrics in a text file to use it in the LSTM RNN 

In [21]:
# Save Lyrics in .txt file
with open('lyricsText.txt', 'w',encoding="utf-8") as filehandle:  
    for listitem in lyrics:
        filehandle.write('%s\n' % listitem)


After getting the wanted data from the dataset , we need to preprocess it.

## Preprocessing The Lyrics




### 1- Convert the lyrics to lowercase :


In [ ]:
# Load the dataset and convert it to lowercase :
textFileName = 'lyricsText.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()
raw_text = raw_text.lower()


### 2- Mapping characters :
 Make two dictionaries , one to convert chars to ints , the other to convert ints back to chars : 


In [ ]:
# Mapping chars to ints :
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))


In [ ]:
# Get number of chars and vocab in our text :
n_chars = len(raw_text)
n_vocab = len(chars)


In [ ]:
print('Total Characters : ' , n_chars) # number of all the characters in lyricsText.txt
print('Total Vocab : ', n_vocab) # number of unique characters


Total Characters :  173698
Total Vocab :  58


### 3- Make samples and labels :
Make samples and labels to feed the LSTM RNN

In [ ]:
# process the dataset:
seq_len = 100
data_X = []
data_y = []

for i in range(0, n_chars - seq_len, 1):
    # Input Sequeance(will be used as samples)
    seq_in  = raw_text[i:i+seq_len]
    # Output sequence (will be used as target)
    seq_out = raw_text[i + seq_len]
    # Store samples in data_X
    data_X.append([chars_int[char] for char in seq_in])
    # Store targets in data_y
    data_y.append(chars_int[seq_out])
n_patterns = len(data_X)
print( 'Total Patterns : ', n_patterns)


Total Patterns :  173598


### 4- Prepare the samples and labels :
prepare the samples and labels to be ready to go into our model.
* Reshape the samples
* Normalize them
* One hot encode the output targets 

In [ ]:
# Reshape X to be suitable to go into LSTM RNN :
X = np.reshape(data_X , (n_patterns, seq_len, 1))
# Normalizing input data :
X = X/ float(n_vocab)
# One hot encode the output targets :
y = np_utils.to_categorical(data_y)

After we finished processing the dataset , we will start building our LSTM RNN model .

## Building The Model 

## First way : From Scratch 

We will start by determining how many layers our model will has , and how many nodes each layer will has :

In [ ]:
LSTM_layer_num = 4 # number of LSTM layers
layer_size = [256,256,256,256] # number of nodes in each layer

Define a sequential model :


In [ ]:
model = Sequential()

### LSTM layer VS CuDNNLSTM layer 
The main difference is that LSTM uses the CPU and CuDNNLSTM uses the GPU , that's why CuDNNLSTM is much faster than LSTM , it is x15 faster.

This is the reason that made me use CuDNNLTSM instead of LSTM .

**Note :** make sure to change the runtime setting of colab to use its GPU .

Add an input layer :

In [ ]:
model.add(CuDNNLSTM(layer_size[0], input_shape =(X.shape[1], X.shape[2]), return_sequences = True))

Add some hidden layers : 

In [ ]:
for i in range(1,LSTM_layer_num) :
    model.add(CuDNNLSTM(layer_size[i], return_sequences=True))

Flatten the data that is coming from the last hidden layer to input it to the output layer :

In [ ]:
model.add(Flatten())

Add an output layer and define its activation function to be **'softmax'** 

and then compile the model with the next params :
*  loss = 'categorical_crossentropy'
*  optimizer = 'adam'

In [ ]:
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

Print a summary of the model to see some details :

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100, 256)          265216    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 100, 256)          526336    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 100, 256)          526336    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 100, 256)          526336    
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 58)                1484858   
_________________________________________________________________
activation_1 (Activation)    (None, 58)                0         
Total para

After we defined the model , we will define the needed callbacks.

### What is a callback ?
A callback is a function that is called after every epoch

in our case we will call the checkpoint callback , what a checkpoint callback does is saving the weights of the model every time the model gets better.

In [ ]:
# Configure the checkpoint :
checkpoint_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

## Training 
A model can't do a thing if it did not train.

As they say **"No train no gain "**

Feel free to tweak `model_params` to get a better model

In [ ]:
# Fit the model :
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callbacks_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle': True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,
          y,
          epochs = model_params['epochs'],
           batch_size = model_params['batch_size'],
           callbacks= model_params['callbacks'],
           verbose = model_params['verbose'],
           validation_split = model_params['validation_split'],
           validation_data = model_params['validation_data'],
           shuffle = model_params['shuffle'],
           initial_epoch = model_params['initial_epoch'],
           steps_per_epoch = model_params['steps_per_epoch'],
           validation_steps = model_params['validation_steps'])


Train on 138878 samples, validate on 34720 samples
Epoch 1/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.8300 - val_loss: 2.7864

Epoch 00001: loss improved from 3.00537 to 2.82996, saving model to Weights-LSTM-improvement-001-2.82996-bigger.hdf5
Epoch 2/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.6424 - val_loss: 2.6723

Epoch 00002: loss improved from 2.82996 to 2.64236, saving model to Weights-LSTM-improvement-002-2.64236-bigger.hdf5
Epoch 3/30
138878/138878 [==============================] - 186s 1ms/step - loss: 2.3721 - val_loss: 2.5978

Epoch 00003: loss improved from 2.64236 to 2.37208, saving model to Weights-LSTM-improvement-003-2.37208-bigger.hdf5
Epoch 4/30
138878/138878 [==============================] - 186s 1ms/step - loss: 1.9650 - val_loss: 2.6343

Epoch 00004: loss improved from 2.37208 to 1.96500, saving model to Weights-LSTM-improvement-004-1.96500-bigger.hdf5
Epoch 5/30
138878/138878 [===================

We can see that some files have been downloaded, we can use such files to load the trained weights to be used in untrained models (i.e we don't have to train a model every time we want to use it) 

### How to Load the Weights ?


In [ ]:
# Load wights file :
wights_file = './models/Weights-LSTM-improvement-004-2.49538-bigger.hdf5' # weights file path
model.load_weights(wights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

Now , after we trained the model ,we can use it to generate fake Taylor Swift lyrics 

## Generating lyrics 
We first pick a random seed , then we will use it to generate lyrics character by character .

In [ ]:
# set a random seed :
start = np.random.randint(0, len(data_X)-1)
pattern = data_X[start]
print('Seed : ')
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")

# How many characters you want to generate
generated_characters = 300

# Generate Charachters :
for i in range(generated_characters):
    x = np.reshape(pattern, ( 1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x,verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    #seq_in = [int_chars[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print('\nDone')


Seed : 
"  once, i've been waiting, waiting
ooh whoa, ooh whoa
and all at once, you are the one, i have been w "

eu h mool shoea
a eir, bo ly lean on the sast
is tigm's the noen uo doy, fo shey stant tas you fot you srart aoo't you tein so my liost
i spaye 
somethppel' cua
iy yas tn mu, io' me
ohehip in the uorlirs tiines ho a ban't teit dven aester, tee tame
mnweiny you'd be pe k bet thing
oe eowt the light i
Done


You might noticed that the generated lyrics are not real , and there are many spelling mistakes.

You can tweak some parameters and add a Dropout layer to avoid overfitting ,then the model could be better at generating tolerable lyrics.

but if you are lazy and don't want to bother yourself trying these steps , try using [textgenrnn](https://goo.gl/E7szXj)

## Second way : Using [textgenrnn](https://goo.gl/E7szXj)


### Import the dependencies :

In [ ]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
import os

### Configure the model :

In [ ]:
model_cfg = {
    'rnn_size': 500,
    'rnn_layers': 12,
    'rnn_bidirectional': True,
    'max_length': 15,
    'max_words': 10000,
    'dim_embeddings': 100,
    'word_level': False,
}

train_cfg = {
    'line_delimited': True,
    'num_epochs': 100,
    'gen_epochs': 25,
    'batch_size': 750,
    'train_size': 0.8,
    'dropout': 0.0,
    'max_gen_length': 300,
    'validation': True,
    'is_csv': False
}

### Upload the dataset :

In [ ]:
uploaded = files.upload()
all_files = [(name, os.path.getmtime(name)) for name in os.listdir()]
latest_file = sorted(all_files, key=lambda x: -x[1])[0][0]

### Train the model

In [ ]:
model_name = '500nds_12Lrs_100epchs_Model'
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=latest_file,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=train_cfg['batch_size'],
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    max_gen_length=train_cfg['max_gen_length'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=model_cfg['dim_embeddings'],
    word_level=model_cfg['word_level'])

In [ ]:
print(textgen.model.summary())

### Download the trained weights :

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

### Load trained model and use it :

In [ ]:
textgen = textgenrnn(weights_path='6layers30EpochsModel_weights.hdf5',
                       vocab_path='6layers30EpochsModel_vocab.json',
                       config_path='6layers30EpochsModel_config.json')

generated_characters = 300

textgen.generate_samples(300)
textgen.generate_to_file('lyrics.txt', 300)

Some lyrics generated by a model created using   [textgenrnn](https://goo.gl/E7szXj) :


```

i ' m not your friends
and it rains when you ' re not speaking
but you think tim mcgraw
and i ' m pacing down
i ' m comfortable
i ' m not a storm in mind
you ' re not speaking
and i ' m not a saint and i ' m standin ' t know you ' re
i ' m wonderstruck
and you ' re gay

i ' ve been giving out
but i ' m just another picture to pay
you ' re not asking myself , oh , i ' d go back to december , don ' t know you
it ' s killing me like a chalkboard
it ' s the one you
can ' t you ' re jumping into of you ' re not a last kiss
and i ' m just a girl , baby , i ' m alone for me
i ' m not a little troubling

won ' t you think about a . steps , you roll the stars mind
you ' s killing me ? )
and i ' m say i won ' t stay beautiful at onto the first page
you ' s 2 : pretty
and you said real
?
change makes and oh , who baby , oh , and you talk away
and you ' s all a minute , ghosts your arms page
these senior making me tough , so hello growing up , we were liar , no one someone perfect day when i came
' re not sorry
you ' re an innocent
on the outskirts

ight , don ' t say a house and he ' round
she ' re thinking to december all that baby , all everything now
and let me when you oh , what to come back my dress
always
i close both young before
at ?
yeah
```



We saw how easy and convenient it was using  [textgenrnn](https://goo.gl/E7szXj) , yes the lyrics still not realistic, but there are much less spelling mistakes than the model that we built from scratch.

another good thing about  [textgenrnn](https://goo.gl/E7szXj) is that one don't have to deal with any dataset processing, just upload the text dataset and set down with a cup of coffee watching your model training and getting better 

## Next Steps :
Now, after you learned how to make a LSTM RNN from scratch to generate texts , and also how to use Pyhton modules such as [textgenrnn](https://goo.gl/E7szXj) you can do many things using this knowledge :
* Try to use other datasets (wikipedia articles , William Shakespeare novevls, etc) to generate novels or articles.
* Use  LSTM RNN in other applications than text generating .
* Read more about LSTM RNN

## References :
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://goo.gl/Kbpk8S)
* [Applied Introduction to LSTMs with GPU for text generation](https://goo.gl/xnQSJU)
* [Generating Text Using LSTM RNN](https://goo.gl/7GxTxu)
* [textgenrnn](https://goo.gl/E7szXj)
* [Train a Text-Generating Neural Network for Free with textgenrnn](https://goo.gl/biaFCr)
* [Understanding LSTM Networks](https://goo.gl/dgmxQm)
* [Long short-term memory](https://en.wikipedia.org/wiki/Long_short-term_memory)